In [2]:
import pandas as pd
import datetime
import csv
import numpy as np
import matplotlib.image
import matplotlib.pyplot as plt
import gmaps
import gmaps.datasets

In [3]:
tashu_csv = pd.read_csv('./tashu.csv', dtype={'RENT_DATE': str, 'RETURN_DATE': str})
station_csv = pd.read_csv('./station.csv')
#NA값이 있는 행을 전부 없앤다.
tashu_csv = tashu_csv.dropna()

tashu_csv['RENT_DATE'] = pd.to_datetime(tashu_csv['RENT_DATE'], format = '%Y%m%d%H%M%S')
tashu_csv['RETURN_DATE'] = pd.to_datetime(tashu_csv['RETURN_DATE'], format = '%Y%m%d%H%M%S')

In [168]:
tashu_csv[:10]

,RENT_STATION,RENT_DATE,RETURN_STATION,RETURN_DATE
0,43.0,2013-01-01 05:56:03,34.0,2013-01-01 06:02:17
2,2.0,2013-01-01 06:04:06,10.0,2013-01-01 06:18:59
3,106.0,2013-01-01 10:53:05,105.0,2013-01-01 10:57:43
4,4.0,2013-01-01 11:22:23,4.0,2013-01-01 12:17:53
5,21.0,2013-01-01 11:39:53,105.0,2013-01-01 11:49:43
6,90.0,2013-01-01 12:08:33,91.0,2013-01-01 12:51:36
7,13.0,2013-01-01 13:14:29,30.0,2013-01-01 13:30:39
8,1.0,2013-01-01 13:37:42,1.0,2013-01-01 13:38:15
9,1.0,2013-01-01 13:38:13,2.0,2013-01-01 15:09:58
10,1.0,2013-01-01 13:38:47,2.0,2013-01-01 15:10:14


In [4]:
#반납한 시간 - 빌린 시간으로 얼마나 빌렸는지 시간을 알 수 있다.
tashu_csv['SUB'] = tashu_csv.RETURN_DATE - tashu_csv.RENT_DATE
tashu_csv['SUB'].astype('timedelta64[s]')

0           374.0
2           893.0
3           278.0
4          3330.0
5           590.0
6          2583.0
7           970.0
8            33.0
9          5505.0
10         5487.0
11         2239.0
12           28.0
13         4081.0
14          379.0
15         1220.0
16         1704.0
17          350.0
18          327.0
19         1156.0
20         4355.0
21         4325.0
22          546.0
23          191.0
24          171.0
25           87.0
26           24.0
27          542.0
28         3659.0
29          873.0
30          763.0
            ...  
3404633    1191.0
3404634     729.0
3404635     211.0
3404636     576.0
3404637     813.0
3404638     332.0
3404639     369.0
3404640    1954.0
3404641      18.0
3404642     615.0
3404643    1657.0
3404644    1717.0
3404645    3520.0
3404646     358.0
3404647     913.0
3404648     270.0
3404649     383.0
3404650     664.0
3404651    2541.0
3404652     307.0
3404653     558.0
3404654     740.0
3404655    1607.0
3404656     548.0
3404657   

In [5]:
#(0 ~ 6 월 ~ 일) weekday 시간별로 날짜 별로 전부 만들어 준다. 월별로 일별로
tashu_csv['weekday'] = pd.DatetimeIndex(tashu_csv['RENT_DATE']).weekday
tashu_csv['weekday_name'] = pd.DatetimeIndex(tashu_csv['RENT_DATE']).weekday_name
tashu_csv['year'] = pd.DatetimeIndex(tashu_csv['RENT_DATE']).year
tashu_csv['month'] = pd.DatetimeIndex(tashu_csv['RENT_DATE']).month
tashu_csv['day'] = pd.DatetimeIndex(tashu_csv['RENT_DATE']).day
tashu_csv['hour'] = pd.DatetimeIndex(tashu_csv['RENT_DATE']).hour
tashu_csv['week'] = pd.DatetimeIndex(tashu_csv['RENT_DATE']).week
tashu_csv

,RENT_STATION,RENT_DATE,RETURN_STATION,RETURN_DATE,SUB,weekday,weekday_name,year,month,day,hour,week
0,43.0,2013-01-01 05:56:03,34.0,2013-01-01 06:02:17,00:06:14,1,Tuesday,2013,1,1,5,1
2,2.0,2013-01-01 06:04:06,10.0,2013-01-01 06:18:59,00:14:53,1,Tuesday,2013,1,1,6,1
3,106.0,2013-01-01 10:53:05,105.0,2013-01-01 10:57:43,00:04:38,1,Tuesday,2013,1,1,10,1
4,4.0,2013-01-01 11:22:23,4.0,2013-01-01 12:17:53,00:55:30,1,Tuesday,2013,1,1,11,1
5,21.0,2013-01-01 11:39:53,105.0,2013-01-01 11:49:43,00:09:50,1,Tuesday,2013,1,1,11,1
6,90.0,2013-01-01 12:08:33,91.0,2013-01-01 12:51:36,00:43:03,1,Tuesday,2013,1,1,12,1
7,13.0,2013-01-01 13:14:29,30.0,2013-01-01 13:30:39,00:16:10,1,Tuesday,2013,1,1,13,1
8,1.0,2013-01-01 13:37:42,1.0,2013-01-01 13:38:15,00:00:33,1,Tuesday,2013,1,1,13,1
9,1.0,2013-01-01 13:38:13,2.0,2013-01-01 15:09:58,01:31:45,1,Tuesday,2013,1,1,13,1
10,1.0,2013-01-01 13:38:47,2.0,2013-01-01 15:10:14,01:31:27,1,Tuesday,2013,1,1,13,1


In [174]:
heatdata = tashu_csv[(tashu_csv.year == 2015)]
heatdata

,RENT_STATION,RENT_DATE,RETURN_STATION,RETURN_DATE,SUB,weekday,weekday_name,year,month,day,hour,week
2236801,31.0,2015-01-01 00:02:52,2.0,2015-01-03 16:29:43,2 days 16:26:51,3,Thursday,2015,1,1,0,1
2236802,121.0,2015-01-01 00:11:25,121.0,2015-01-01 00:11:42,0 days 00:00:17,3,Thursday,2015,1,1,0,1
2236803,121.0,2015-01-01 00:11:44,121.0,2015-01-01 00:12:01,0 days 00:00:17,3,Thursday,2015,1,1,0,1
2236804,112.0,2015-01-01 05:03:20,138.0,2015-01-01 09:18:30,0 days 04:15:10,3,Thursday,2015,1,1,5,1
2236805,144.0,2015-01-01 06:58:18,144.0,2015-01-01 06:58:44,0 days 00:00:26,3,Thursday,2015,1,1,6,1
2236806,144.0,2015-01-01 06:58:45,144.0,2015-01-01 06:59:06,0 days 00:00:21,3,Thursday,2015,1,1,6,1
2236807,144.0,2015-01-01 06:59:09,144.0,2015-01-01 06:59:33,0 days 00:00:24,3,Thursday,2015,1,1,6,1
2236808,144.0,2015-01-01 07:03:21,144.0,2015-01-01 07:03:52,0 days 00:00:31,3,Thursday,2015,1,1,7,1
2236809,21.0,2015-01-01 07:04:01,32.0,2015-01-01 07:15:18,0 days 00:11:17,3,Thursday,2015,1,1,7,1
2236810,2.0,2015-01-01 07:47:28,2.0,2015-01-01 07:47:57,0 days 00:00:29,3,Thursday,2015,1,1,7,1


In [175]:
df = pd.DataFrame(heatdata)
filename = 'heatdata.csv'
df.to_csv(filename, index=False, encoding='utf-8')

In [20]:
station_csv[:10]

,번호,키오스크번호,구별,명칭,위치,주소,거치대,좌표
0,1,1,유성구,무역전시관입구(택시승강장 앞),엑스포다리 맞은편,유성구 도룡동 3-8,14,"36.374325,127.387462"
1,2,2,유성구,대전컨벤션 센터 앞,둔산대교 맞은편,유성구 도룡동 4-19,20,"36.374472,127.392241"
2,3,3,서구,한밭수목원(정문입구),한밭수목원 내,서구 만년동 396,19,"36.369855,127.388749"
3,4,4,서구,초원아파트104동부근(버스정류장),초원아파트 104동앞 쪽문 육교 버스정류장 앞,서구 만년동 401,12,"36.368192,127.379281"
4,5,5,서구,둔산대공원 입구(버스정류장),한밭수목원에서 평송수련원 가는길 버스정류장 앞,서구 둔산동 1521-10,13,"36.365034,127.389361"
5,6,6,서구,백합4가 앞(농협앞),백합아파트 상가 \n농협 버스정류장 앞,서구 월평2동 266,12,"36.362304,127.376419"
6,7,7,서구,정부청사 입구(대덕대로),둔산 시외버스터미널 버스정류장 앞,서구 둔산동 920-2,13,"36.361665,127.379699"
7,8,8,서구,정부청사 입구(샘머리),둔산 고속버스터미널 버스정류장 앞,서구 둔산동 1518,12,"36.361794,127.390417"
8,9,9,서구,황실아파트앞(성룡초교 앞),성룡초교 정문 \n버스정류장앞,서구 월평동 304,12,"36.361392,127.374236"
9,10,10,서구,만년동 KBS 부근(기업은행 앞),초원아파트 102동 \n육교 건너편,서구 만년동 300,12,"36.369207,127.379801"


In [155]:
#inplace => sorting 한 결과를 그 값 자체에 저장하는것
#경로 top20을 구해낸다.
trace_df = pd.DataFrame({'count' : tashu_csv.groupby(["RENT_STATION", "RETURN_STATION"]).size()}).reset_index()
trace_df.sort_values('count', ascending=False,inplace=True)
top20_trace = trace_df[:20]

top20_trace

,RENT_STATION,RETURN_STATION,count
342,3.0,3.0,84496
4873,31.0,31.0,21749
8708,56.0,56.0,18343
3361,21.0,105.0,17220
0,1.0,1.0,14489
5045,32.0,32.0,12177
15442,105.0,21.0,12154
5218,33.0,33.0,11973
2625,17.0,17.0,11966
8684,56.0,32.0,11868


In [156]:
top20_trace['rent_latlon'] = 0
top20_trace['return_latlon'] = 0

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [157]:
#경로 top20의 rent와 return의 좌표를 저장한다.
for i, row in top20_trace.iterrows():
    top20_trace.loc[i,'rent_latlon'] = station_csv['좌표'][row['RENT_STATION'] - 1]
    top20_trace.loc[i,'return_latlon'] = station_csv['좌표'][row['RETURN_STATION'] - 1]
top20_trace    

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,RENT_STATION,RETURN_STATION,count,rent_latlon,return_latlon
342,3.0,3.0,84496,"36.369855,127.388749","36.369855,127.388749"
4873,31.0,31.0,21749,"36.361773, 127.357485","36.361773, 127.357485"
8708,56.0,56.0,18343,"36.361736, 127.344994","36.361736, 127.344994"
3361,21.0,105.0,17220,"36.373457,127.359293","36.364177,127.358845"
0,1.0,1.0,14489,"36.374325,127.387462","36.374325,127.387462"
5045,32.0,32.0,12177,"36.359293, 127.354503","36.359293, 127.354503"
15442,105.0,21.0,12154,"36.364177,127.358845","36.373457,127.359293"
5218,33.0,33.0,11973,"36.358494, 127.361197","36.358494, 127.361197"
2625,17.0,17.0,11966,"36.35219,127.378814","36.35219,127.378814"
8684,56.0,32.0,11868,"36.361736, 127.344994","36.359293, 127.354503"


In [138]:
#top10의 lat과 lon을 분리 시킨다
for index, station in top10_trace.iterrows():
    rent_location = station['rent_latlon'].split(',')
    return_location = station['return_latlon'].split(',')
    print(rent_location, return_location)

['36.368192', '127.379281'] ['36.368192', '127.379281']
['36.359293', ' 127.354503'] ['36.359293', ' 127.354503']
['36.357928', ' 127.342792'] ['36.357928', ' 127.342792']
['36.370263', '127.362136'] ['36.367989', '127.356852']
['36.374472', '127.392241'] ['36.374472', '127.392241']
['36.358494', ' 127.361197'] ['36.358494', ' 127.361197']
['36.367989', '127.356852'] ['36.370263', '127.362136']
['36.357889', ' 127.365972'] ['36.357889', ' 127.365972']
['36.351132', '127.384838'] ['36.351132', '127.384838']
['36.357928', ' 127.342792'] ['36.358494', ' 127.361197']
['36.358494', ' 127.361197'] ['36.357928', ' 127.342792']
['36.361736', ' 127.344994'] ['36.361736', ' 127.344994']
['36.362304', '127.376419'] ['36.362304', '127.376419']
['36.342378', '127.387671'] ['36.342378', '127.387671']
['36.316915', ' 127.392467'] ['36.316915', ' 127.392467']
['36.361392', '127.374236'] ['36.361392', '127.374236']
['36.355591', '127.390063'] ['36.355591', '127.390063']
['36.370263', '127.362136'] ['36

In [158]:
#만든 top20 DataFrame형태를 csv형태로 뽑아낸다.
df = pd.DataFrame(top20_trace)
filename = 'top20_trace.csv'
df.to_csv(filename, index=False, encoding='utf-8')

In [6]:
df = pd.DataFrame(tashu_csv)
filename = 'tashu_result.csv'
df.to_csv(filename, index=False, encoding='utf-8')

## 화요일과 목요일의 오전 7 ~ 9 오후 18 ~ 20 에 빌린 사람을 뽑아낸다.

In [144]:
#화요일과 목요일의 오전 7 ~ 9 오후 18 ~ 20 에 빌린 사람을 뽑아낸다.
result = tashu_csv[((tashu_csv.weekday == 1) | (tashu_csv.weekday == 5)) & 
                   ((tashu_csv.hour >= 7) & (tashu_csv.hour <= 9) | (tashu_csv.hour >= 18) & (tashu_csv.hour <= 20))]
result[:10]

,RENT_STATION,RENT_DATE,RETURN_STATION,RETURN_DATE,SUB,weekday,weekday_name,hour
50,50.0,2013-01-01 18:15:22,20.0,2013-01-01 18:19:45,0 days 00:04:23,1,Tuesday,18
51,90.0,2013-01-01 18:38:43,90.0,2013-01-01 18:39:21,0 days 00:00:38,1,Tuesday,18
52,91.0,2013-01-01 18:45:00,41.0,2013-03-03 17:41:31,60 days 22:56:31,1,Tuesday,18
53,64.0,2013-01-01 19:06:52,64.0,2013-01-01 19:09:52,0 days 00:03:00,1,Tuesday,19
54,63.0,2013-01-01 19:16:21,63.0,2013-01-01 19:19:21,0 days 00:03:00,1,Tuesday,19
55,8.0,2013-01-01 19:31:30,8.0,2013-01-01 19:34:30,0 days 00:03:00,1,Tuesday,19
56,34.0,2013-01-01 19:45:04,34.0,2013-01-01 19:46:58,0 days 00:01:54,1,Tuesday,19
57,7.0,2013-01-01 19:51:50,31.0,2013-01-01 20:26:09,0 days 00:34:19,1,Tuesday,19
58,106.0,2013-01-01 19:52:37,106.0,2013-01-01 19:52:52,0 days 00:00:15,1,Tuesday,19
59,67.0,2013-01-01 20:27:41,68.0,2013-01-01 20:36:33,0 days 00:08:52,1,Tuesday,20


In [146]:
#화 목요일 의 경로 top20
result_df = pd.DataFrame({'count' : result.groupby(["RENT_STATION", "RETURN_STATION"]).size()}).reset_index()
result_df.sort_values('count', ascending=False,inplace=True)
top20_result = result_df[:20]
top20_result

,RENT_STATION,RETURN_STATION,count
271,3.0,3.0,8028
3510,31.0,31.0,2478
0,1.0,1.0,2015
6249,56.0,56.0,1864
1960,17.0,17.0,1524
3656,32.0,32.0,1405
2514,21.0,105.0,1323
3787,33.0,33.0,1280
899,8.0,8.0,1259
560,5.0,5.0,1248


In [159]:
top20_result['rent_latlon'] = 0
top20_result['return_latlon'] = 0

for i, row in top20_result.iterrows():
    top20_result.loc[i,'rent_latlon'] = station_csv['좌표'][row['RENT_STATION']-1]
    top20_result.loc[i,'return_latlon'] = station_csv['좌표'][row['RETURN_STATION']-1]
top20_result  

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.


,RENT_STATION,RETURN_STATION,count,rent_latlon,return_latlon
271,3.0,3.0,8028,"36.369855,127.388749","36.369855,127.388749"
3510,31.0,31.0,2478,"36.361773, 127.357485","36.361773, 127.357485"
0,1.0,1.0,2015,"36.374325,127.387462","36.374325,127.387462"
6249,56.0,56.0,1864,"36.361736, 127.344994","36.361736, 127.344994"
1960,17.0,17.0,1524,"36.35219,127.378814","36.35219,127.378814"
3656,32.0,32.0,1405,"36.359293, 127.354503","36.359293, 127.354503"
2514,21.0,105.0,1323,"36.373457,127.359293","36.364177,127.358845"
3787,33.0,33.0,1280,"36.358494, 127.361197","36.358494, 127.361197"
899,8.0,8.0,1259,"36.361794,127.390417","36.361794,127.390417"
560,5.0,5.0,1248,"36.365034,127.389361","36.365034,127.389361"


In [160]:
df = pd.DataFrame(top20_result)
filename = 'top20_result.csv'
df.to_csv(filename, index=False, encoding='utf-8')